# Ch_07_합성곱 신경망

딥 러닝의 활용 분야 중 특히, 이미지 인식 분야에서 가장 많이 활용되고 있는 CNN에 대해서 살펴보자.

7.1 전체 구조

CNN은 이전의 신경망에서 합성곱 계층(Convolutional Layer)과 풀링 계층(Pooling Layer)이란 개념이 등장. 이 개념 중심으로 CNN을 이해해보자.

지금까지의 신경망은, 한 개의 데이터가 모든 뉴런과 연결(이를 완전연결,Affine 계층이라 부름) 되어 층을 지나 출력층으로 나타났다. 그림으로 보면,

[그림 7-1] 완전 연결으로 이루어진 신경망
![](data/images/fig%207-1.png)
하지만 CNN은 활성화 함수 전 후 단계에서 합성곱 계층과 풀링 계층이란 단계를 거치게 된다.

[그림 7-2] CNN 신경망
![](data/images/fig%207-2.png)

7.2 합성곱 계층

그렇다면 활성화 함수 전 단계인 합성곱 계층에선 무슨 일이 일어나는지 살펴보자.

7.2.1 완전연결 계층의 문제점

합성곱 계층이 왜 나오게 되었는지 완전연결 계층에서 어떤 문제가 생기는지 살펴보자.

완전연결 계층의 신경망에서는 데이터의 형상이 무시 된다. 이미지 데이터를 예로 들면, 이미지는 가로,세로,색상의 3개의 feature를 가지는데, 완전연결 계층의 신경망에서는 이를 단순히 1열의 숫자들의 배열로 나타냄으로써 이미지가 가지는 공간적 정보가 소멸이 된다.

예를 들면, 공간적으로 가까운 픽셀은 비슷한 색상을 낼텐데 이를 하나의 벡터로 나타냄으로써 그런 공간적 가까움이 없어진다.

이런 문제점을 해결하기 위해 고안 된 것이 바로 합성곱 계층이다.

7.2.2 합성곱 연산

합성곱 연산은 필터라는 녀석을 통해 연산이 되는데, 그림을 보자.

[그림 7-3] 합성곱 연산의 예
![](data/images/fig%207-3.png)
완전연결 계층과 달리, CNN에서는 데이터가 행렬(혹은 텐서)의 형태 그대로 입력이 되고, 이에 필터라는 것과 연산이 된다.

때에 따라서는 필터를 커널이라고 하기도 한다.

[그림 7-4] 합성곱 연산 작동 방법
![](data/images/fig%207-4.png)
합성곱은 필터와 같은 크기의 윈도우(Window)를 일정 간격으로 이동시켜가며 행렬의 원소별 곱을 한 후 다 더한 값을 출력으로 갖는다.

완전연결 계층에서와는 달리 여기서는 이 필터의 값이 바로 가중치 매개변수가 된다.

그렇다면 완전연결 계층에서의 편향은 CNN에서 어떻게 고려될까?

[그림 7-5] 합성곱 연산의 편향
![](data/images/fig%207-5.png)
하나의 편향이 출력 값 모두에 더해진다.

7.2.3 패딩(Padding)

위에서 보듯이 입력데이터의 크기가 $4\times4$ 이고, 필터가 $3\times3$이기 때문에, 출력 데이터가 $2\times2$가 된다. 만약 이렇게 계속 층을 지난다면 입력층에 비해 출력층의 데이터의 사이즈가 점점 작아지고 시작을 아무리 크게해도 층을 지나면서 출력된 데이터의 크기가 작아진다.

이를 보완하기 위해서 입력층의 테두리에 0으로 데이터를 늘리는데, 이런 작업을 패딩(padding)이라고 한다. 

[그림 7-6] 폭이 1인 패딩을 입력 데이터에 적용
![](data/images/fig%207-6.png)

7.2.4 스트라이드(Stride, 동사: 성큼성큼 걷다)

[그림 7-4]에서 입력데이터에 필터의 윈도우를 적용시킬 때, 1칸씩 움직이며 적용시켰다. 만약 이 위치를 2칸,3칸을 하면 어떻게 될까?

필터를 적용하는 위치의 간격을 스트라이드,stride라고 한다.

[그림 7-7] 스트라이드가 2인 합성곱 연산
![](data/images/fig%207-7.png)
그러면 지금까지 고려된 패딩, 스트라이드의 크기에 따라 출력 데이터가 어떻게 되는지 살펴보자.

입력 데이터의 크기: $(H,W)$

필터의 크기: $(FH, FW)$

패딩 : P

스트라이드 : S

출력 데이터의 크기: $(OH, OW)$

[식 7.1] 패딩,스트라이드에 따른 출력 데이터의 크기
![](data/images/e%207.1.png)
Ex) [그림 7-6]

입력:(4,4), 패딩:1, 스트라이드:1, 필터:(3,3)

$ OH = \frac{4+2 \cdot 1 -3}{1} +1 =4 $

$ OW = \frac{4+2 \cdot 1 -3}{1} +1 =4 $

단, 코딩을 할 때 정수가 나오지 않는 경우, 반올림을 하는 등을 통해 오류를 예방해야겠다.

7.2.5 3차원 데이터의 합성곱 연산

지금까지 2차원 데이터의 합성곱 연산에 대해 알아보았다. 실제로 이미지는 가로,세로에다가 채널이라는 feature가 고려되므로 3차원의 데이터 형태를 나타낸다.

직관적으로 3차원 데이터가 들어왔을 때는, 3차원의 수만큼 필터가 늘어나면 될것 같다.

[그림 7-8] 3차원 데이터의 합성곱 연산
![](data/images/fig%207-8.png)
여기서 주의 할 것은, 출력데이터의 차원은 여전히 2차원이라는 점이다. 더 자세히 연산의 과정을 살펴보자.

[그림 7-9] 3차원 데이터의 합성곱 연산 순서
![](data/images/fig%207-9.png)

7.2.6 블록으로 생각

3차원의 데이터를 하나의 블록으로 생각해보자.3차원의 데이터를 블록으로 나타낼 때 (채널,높이,너비)순으로 나타내도록 하자.

[그림 7-10] 입력 데이터가 블록인 합성곱 연산
![](data/images/fig%207-10.png)
이 결과 출력 데이터는 채널이 1인 데이터가 나온다. 만약 다수의 채널을 가진 출력 데이터를 갖고 싶다면, 필터를 늘리면 된다.

[그림 7-11] 여러개의 필터를 이용한 합성곱 연산
![](data/images/fig%207-11.png)
이 단계를 거쳐 데이터의 크기가 어느정도 유지된 채 층을 흘려보내는 것이 CNN이다.

이제 다시 편향을 고려하면,

[그림 7-12] 합성곱 연산에서의 편향
![](data/images/fig%207-12.png)

따라서, CNN의 가중치 매개변수는 필터에서 (출력 채널 수, 입력 채널 수, 높이, 너비)의 4차원 데이터를 갖게 되고, 편향은 (출력 채널수,1,1)인 3차원 데이터를 갖게 된다.

7.2.7 배치처리

지금까지는 하나의 데이터의 CNN에 대해서 살펴보았다. 하지만, 컴퓨터는 하나의 계산보다는 배치의 계산에 좀 더 특화 되어있다고 설명했으므로, CNN에도 배치로 데이터를 처리하려고 한다. 완전연결에서와 마찬가지로, 입력 데이터를 배치 크기로 묶어서 CNN으로 흘려보내는 것이다. 
CNN에서는 데이터가 3차원으로 나타나지므로, 배치 데이터를 시각화 할 수는 없지만 기본적인 원리는 같다.

[그림 7-13] 배치의 CNN 연산 처리
![](data/images/fig%207-13.png)
한 번에 N개의 데이터가 동시에 층을 흐른다고 생각하면 된다.

7.3 풀링(Pooling) 계층

앞에서 CNN은 합성곱 계층과 풀링 계층이라는 2가지 개념으로 인해 이루어진다고 했다. 지금까지 합성곱 계층의 연산을 보았고 이제 풀링 계층을 보자.

간단하게 이야기하면 풀링은 가로,세로 바향의 공간을 줄이는 것을 의미한다.

[그림 7-14] $2\times2$ 최대 풀링의 처리 순서
![](data/images/fig%207-14.png)
일반적으로 최대 풀링의 사이즈와 스트라이드는 같은 수로 한다.

최대 풀링 말고 영역안의 평균값을 이용하는 평균 풀링도 존재한다. 하지만 최대 풀링을 좀 더 많이 사용하는 듯 보인다.

7.3.1 풀링 계층의 특징

1. 학습해야할 매개변수가 없다.

:대상 영역에서의 최대값 혹은 평균을 구하는 연산이므로 학습 할 것이 없다.

2. 채널 수가 변하지 않는다.

:합성곱 계층에서는 입력데이터의 채널수가 얼마든지 간에 출력데이터의 채널 수는 1이 되었다. 하지만 풀링 계층은 입력 데이터의 채널 수 그대로를 보낸다.

[그림 7-15] 풀링은 채널 수를 바꾸지 않는다.
![](data/images/fig%207-15.png)

3. 입력의 변화에 영향을 적게 받는다.

:입력데이터가 조금 변해도 풀링의 결과는 잘 변하지 않는다.

[그림 7-16] 입력 데이터가 가로로 1만큼 옮겨가도 풀링은 변하지 않는다.
![](data/images/fig%207-16.png)

7.4 합성곱/풀링 계층 구현

7.4.1 4차원 배열

CNN에 흐르는 입력데이터는 4차원 데이터이다. 데이터의 형상이 $(10,1,28,28)$이라면, 10개의 데이터, 1개의 채널, $28\times28$의 행렬 데이터이다.

In [2]:
#4차원 배열의 데이터
import numpy as np

x = np.random.rand(10,1,28,28) #무작위로 데이터 생성
x.shape

(10, 1, 28, 28)

7.4.2 im2col로 데이터 전개

im2col(image to column, 이미지를 행렬로)이라는 함수를 이용해서 4차원의 데이터를 다루려고 한다.

im2col이라는 함수는 3차원 (혹은 4차원도) 데이터를 2차원의 행렬로 바꾸어 주는 연산이다.

[그림 7-17] im2col의 대략적인 작동
![](data/images/fig%207-17.png)
im2col은 필터에 해당하는 영역을 하나의 열로 바꾸어주는 역할을 한다. 

[그림 7-18] 필터 적용 영역에 따른 im2col의 작동원리
![](data/images/fig%207-18.png)
im2col을 이용하여 입력 데이터에서 필터가 적용되는 부분을 하나의 행으로 만든 후 필터에 해당하는 녀석을 열로 만들어서 필터연산을 수행한다.

[그림 7-19] 합성곱 연산의 펄터 처리 과정
![](data/images/fig%207-19.png)

7.4.3 합성곱 계층 구현

im2col함수는 입력 데이터의 차원, 필터의 높이, 필터의 너비, 스트라이드, 패딩을 인수로 갖는다.


#im2col함수

import sys, os
sys.path.append(os.pardir)
from common.util import im2col

x1 = np.random.rand(1,3,7,7) #(데이터수, 채널수, 높이, 너비)
col1 = im2col(x1,5,5,stride=1,pad=0)
print(col1.shape)

In [4]:
#CNN구현

class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self,x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1+(H+2*self.pad-FH) / self.stride)
        out_w = int(1+(W+2*self.pad-FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN,-1).T # 필터를 벡터로 전개??
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0,3,1,2)
        
        return out

7.6 CNN 시각화

CNN의 합성곱 계층들은 입력 데이터의 어떤 부분을 보는 것인지 살펴보자.

7.6.1 1번째 층의 가중치 시각화

앞서 실습 한 것에, 1번째 층의 합성곱 계층의 가중치의 형상이 (30,1,5,5) 라고 하자.(필터가 30개, 채널1개, 가로,세로5인 필터)

이 가중치들을 다시 이미지로 나타내어 보면, 이 필터들이 입력데이터의 어떤 부분을 보려고 하는지를 알 수 있다.

[그림 7-24] 학습 전과 후의 1번째 합성곱 계층의 가중치
![](data/images/fig%207-24.png)
학습 전의 가중치는 임의로 초기화 되어있어서 아무 규칙도 없지만, 학습 후에는 나름 검정색으로 덩어리 진 부분이나, 경계에 반응하는 필터이다.

[그림 7-25] 가로에지와 세로에지에 반응하는 필터
![](data/images/fig%207-25.png)

7.6.2 층 깊이에 따른 추출 정보 변화

위 [그림 7-25]에서의 출력 이미지가 다음 층으로 가면 갈 수록 어떤 정보가 전달이 될까

논문에 따르면 층에 흐르면 흐를 수록 점점 더 정보가 추상화 되기 때문에 명확하게 말하기는 어렵다.

[그림 7-26] 사물 인식 CNN
![](data/images/fig%207-26.png)

7.7 대표적인 CNN

7.7.1 LeNet

LeNet은 손글씨 인식 네트워크로 1998년에 고안

[그림 7-27] LeNet의 구성
![](data/images/fig%207-27.png)

7.7.2 AlexNet
[그림 7-28] AlexNet
![](data/images/fig%207-28.png)

둘다 비슷해보이지만 차이점이 존재한다.

1. LeNet은 활성화 함수로 시그모이드 함수를, AlexNet은 ReLU함수를 사용

2. AlexNet은 LRN(Local Response Normalization)이라는 국소적 정규화를 실시하는 계층 이용 -> ??

3. AlexNet은 드롭 아웃 사용

7.8 정리

처음에 이야기 했듯이, CNN은 합성곱 계층과 풀링 계층으로 이루어져있다. 각각 계층이 하는일은 생각보다 단순하다.

코드로 구현은 다른 문제일지라도, 각각의 계층에서 하는 개념은 확실히 하자.

